In [1]:
import torch
from transformers import MarianMTModel, MarianTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel
from datasets import load_dataset
import evaluate
from context import summarize_text

In [2]:
dataset_name = "Iker/Document-Translation-en-es"

summary_model_name = "google-t5/t5-small"
translation_model_name = "Helsinki-NLP/opus-mt-en-es"
revision = "main"

tokenizer_model_name = "distilbert-base-uncased"
model_name = "Helsinki-NLP/opus-mt-en-es"

device = 0 if torch.cuda.is_available() else -1

# Get Context

In [3]:
text_file = 'document.txt'

summarize_text(text_file, max_length=35)

C:\Users\uceda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1282: UserWarning: Unfeasible length constraints: `min_length` (40) is larger than the maximum possible length (35). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


'Squirrels are found on every continent except Antarctica and australia. they will stay with their mother for 12 weeks before going off to be independent '

## Base summariser

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(summary_model_name)
tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
text = """
The Hugging Face Transformers library provides state-of-the-art general-purpose architectures
for natural language understanding (NLU) and natural language generation (NLG). These architectures
include BERT, GPT, GPT-2, BART, and T5, which can be applied to text classification, summarization, translation, and more.
"""

inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=100, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

the Hugging Face Transformers library provides state-of-the-art architectures for natural language understanding (NLU) and natural language generation (NLG). these architectures include BERT, GPT, GPT-2, and BART.


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Choose the specific Falcon-Instruct model. For example, "tiiuae/falcon-7b-instruct"
# model_name = "EleutherAI/gpt-neo-1.3B"
model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

prompt = "Translate the following text from English to Spanish: What are the advantages of Deep Learning?"

inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, max_length=50)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

C:\Users\uceda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\uceda\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

C:\Users\uceda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Qué es las ventajas de la energa de Deep Learning?


## Baseline translator

In [8]:
tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
model = MarianMTModel.from_pretrained(translation_model_name)
english_text = "What color is the hrass? Answer in english"
input_ids = tokenizer.encode(english_text, return_tensors="pt")
translated_ids = model.generate(input_ids)

translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

print("Translated Text (Spanish):", translated_text)

Translated Text (Spanish): ¿De qué color es la hrass? Respuesta en inglés


## Load dataset

In [9]:
dataset = load_dataset(dataset_name)
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"].shuffle(seed=42).select(range(100))
test_dataset = split_dataset["test"].shuffle(seed=42).select(range(100))

## x/y split

In [10]:
train_x_data = train_dataset["en"]
train_y_data = train_dataset["es"]
test_x_data = test_dataset["en"]
test_y_data = test_dataset["es"]

In [11]:
len(train_x_data[0])

3100

# Load Pre-Trained Models

In [13]:
model = MarianMTModel.from_pretrained(model_name)
bleu = evaluate.load("sacrebleu")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)

You are using a model of type t5 to instantiate a model of type marian. This is not supported for all configurations of models and can yield errors.


ValueError: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly saved?

## Load tokenizer

In [14]:
# I used the original text (english) to tokenize the data, seemed logical to me, but idk
def tokenize_function(examples):
    return tokenizer(examples["en"], padding="max_length", truncation=True)

train_x_data = train_dataset.map(tokenize_function, batched=True)
test_x_data = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# Evaluate Model

In [15]:
# Okay this maybe works, maybe not, who knows, help me?
input_ids = torch.tensor(test_x_data["input_ids"]).to(model.device)
print('translating the tokens..')
translated_tokens = model.generate(input_ids=input_ids)
predicted_translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

references = [[ref] for ref in test_y_data] #The bleu score needs a list of lists
bleu_score = bleu.compute(predictions=predicted_translations, references=references)

print(f"BLEU score: {bleu_score['score']}")

translating the tokens..


KeyboardInterrupt: 